<a href="https://colab.research.google.com/github/jackschmell/Baseball-AI/blob/main/Jack_Alexnet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from google.colab import drive
from torchsummary import summary 

In [2]:
# 1. Mount Google Drive
drive.mount('/content/gdrive')

# 2. Load labeled images from folders
# data_dir = '/content/gdrive/MyDrive/Data2023/ballgame'
data_dir = '/content/gdrive/MyDrive/Data2023/ballgame'

Mounted at /content/gdrive


In [3]:
# 3. Pre-process the data and create data loaders
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
image_datasets = {x: datasets.ImageFolder(data_dir + '/' + x, data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Load the pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Freeze the weights of the pre-trained model
for param in alexnet.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of pitch types in the dataset
num_pitch_types = 2
alexnet.classifier[6] = torch.nn.Linear(4096, num_pitch_types)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(alexnet.classifier.parameters(), lr=0.001, momentum=0.9)


# Train the model
for epoch in range(10):
    running_loss = 0.0
    for (inputs, labels) in dataloaders["train"]:
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [%d], Loss: %.4f' % (epoch+1, running_loss / len(dataloaders["train"])))

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in dataloaders["valid"]:
        images, labels = data
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))


Epoch [1], Loss: 0.9566
Epoch [2], Loss: 0.8735
Epoch [3], Loss: 0.6814
Epoch [4], Loss: 0.5138
Epoch [5], Loss: 0.6925
Epoch [6], Loss: 0.5052
Epoch [7], Loss: 0.9486
Epoch [8], Loss: 0.5215
Epoch [9], Loss: 0.4533
Epoch [10], Loss: 0.5107
Accuracy of the network on the test images: 81 %


In [ ]:
# Will Improve on accuracy hopefully ! (Include more and better data)
